# 69-性能优化与调试技巧

## 📚 用途说明

**学习目标**：
- 掌握Python性能优化的核心方法和技巧
- 学会使用性能分析工具和调试器
- 理解内存管理和资源优化策略
- 能够建立完整的性能监控体系

**前置要求**：
- 已完成持续集成与持续部署学习
- 熟练掌握Python高级编程技巧
- 了解计算机系统架构和操作系统
- 具备基本的性能分析意识

**与LangChain关联**：
- 支持LangChain应用的性能优化
- 为AI模型推理和训练提供优化方案
- 保障LangChain服务的高性能运行
- 实现AI系统的性能监控和调优

---

## 🔢 知识点覆盖

### 8.5 性能优化与调试技巧 [⭐⭐进阶]
**知识点说明**：性能优化与调试技巧是提升系统性能的关键技能，为LangChain的AI项目提供高效的性能保障方案。

**学习要求**：
- 掌握性能分析和优化方法
- 学会使用调试工具和性能监控
- 理解内存管理和并发优化
- 能够建立完整的性能调优体系

**案例要求**：
- 实现性能分析工具
- 构建调试和监控系统
- 开发优化策略管理器
- 验证点：能独立进行性能分析和优化

In [ ]:
print("⚡ 性能优化与调试技巧:")
print("=" * 50)

# 导入必要的库
import os
import sys
import time
import gc
import psutil
import threading
import multiprocessing
import cProfile
import pstats
import io
import tracemalloc
import linecache
import inspect
import functools
import weakref
import collections
import itertools
import typing
import dataclasses
import uuid
import json
import datetime
from typing import List, Dict, Any, Optional, Union, Tuple, Callable, Type
from dataclasses import dataclass, field, asdict
from enum import Enum
from abc import ABC, abstractmethod
from pathlib import Path

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 性能分析工具
print(f"📝 1. 性能分析工具:")

# 1.1 性能分析器
print(f"\n   🎯 1.1 性能分析器:")

class PerformanceMetricType(Enum):
    """性能指标类型"""
    CPU_TIME = "cpu_time"  # CPU时间
    MEMORY_USAGE = "memory_usage"  # 内存使用
    EXECUTION_TIME = "execution_time"  # 执行时间
    CALL_COUNT = "call_count"  # 调用次数
    CACHE_HIT_RATE = "cache_hit_rate"  # 缓存命中率
    IO_OPERATIONS = "io_operations"  # IO操作
    NETWORK_LATENCY = "network_latency"  # 网络延迟

@dataclass
class PerformanceMetric:
    """性能指标"""
    name: str
    metric_type: PerformanceMetricType
    value: float
    unit: str
    timestamp: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    context: Dict[str, Any] = field(default_factory=dict)
    
    def __str__(self):
        return f"{self.name}: {self.value} {self.unit}"

@dataclass
class FunctionProfile:
    """函数性能分析"""
    function_name: str
    file_name: str
    line_number: int
    call_count: int
    total_time: float
    average_time: float
    cumulative_time: float
    per_call_time: float
    
    @property
    def efficiency_score(self) -> float:
        """效率评分"""
        # 基于调用次数和平均时间的效率评分
        if self.call_count == 0:
            return 0.0
        # 调用次数多但平均时间短的函数效率高
        return min(100.0, (self.call_count / (self.average_time + 0.001)) * 10)

class PerformanceProfiler:
    """性能分析器"""
    
    def __init__(self):
        self.metrics: List[PerformanceMetric] = []
        self.function_profiles: Dict[str, FunctionProfile] = {}
        self.is_profiling = False
        self.start_time: Optional[datetime.datetime] = None
        self.profiler = None
        self.memory_tracker = None
    
    def start_profiling(self):
        """开始性能分析"""
        if self.is_profiling:
            print("性能分析已在运行中")
            return
        
        self.is_profiling = True
        self.start_time = datetime.datetime.utcnow()
        
        # 启动CPU性能分析
        self.profiler = cProfile.Profile()
        self.profiler.enable()
        
        # 启动内存追踪
        tracemalloc.start()
        
        print("性能分析已启动")
    
    def stop_profiling(self) -> Dict[str, Any]:
        """停止性能分析"""
        if not self.is_profiling:
            print("性能分析未在运行")
            return {}
        
        self.is_profiling = False
        end_time = datetime.datetime.utcnow()
        
        # 停止CPU性能分析
        self.profiler.disable()
        
        # 获取内存使用情况
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # 分析性能数据
        stats = pstats.Stats(self.profiler)
        stats.sort_stats('cumulative')
        
        # 提取函数性能数据
        self._extract_function_profiles(stats)
        
        # 收集系统指标
        system_metrics = self._collect_system_metrics()
        
        profile_result = {
            "start_time": self.start_time.isoformat(),
            "end_time": end_time.isoformat(),
            "duration": (end_time - self.start_time).total_seconds(),
            "memory_current": current,
            "memory_peak": peak,
            "function_profiles": self.function_profiles,
            "system_metrics": system_metrics
        }
        
        print(f"性能分析完成，耗时: {profile_result['duration']:.2f}秒")
        return profile_result
    
    def _extract_function_profiles(self, stats: pstats.Stats):
        """提取函数性能数据"""
        self.function_profiles.clear()
        
        for func_info in stats.stats.items():
            func_key, func_stats = func_info
            filename, line_number, function_name = func_key
            
            cc, nc, tt, ct, callers = func_stats
            
            profile = FunctionProfile(
                function_name=function_name,
                file_name=filename,
                line_number=line_number,
                call_count=nc,
                total_time=tt,
                average_time=tt / nc if nc > 0 else 0,
                cumulative_time=ct,
                per_call_time=ct / nc if nc > 0 else 0
            )
            
            key = f"{function_name}:{line_number}"
            self.function_profiles[key] = profile
    
    def _collect_system_metrics(self) -> Dict[str, Any]:
        """收集系统指标"""
        process = psutil.Process()
        
        return {
            "cpu_percent": process.cpu_percent(),
            "memory_percent": process.memory_percent(),
            "memory_rss": process.memory_info().rss,
            "memory_vms": process.memory_info().vms,
            "num_threads": process.num_threads(),
            "open_files": len(process.open_files()),
            "connections": len(process.connections()),
            "system_cpu_percent": psutil.cpu_percent(),
            "system_memory_percent": psutil.virtual_memory().percent
        }
    
    def get_top_functions(self, metric: str = "cumulative_time", limit: int = 10) -> List[FunctionProfile]:
        """获取性能最差的函数"""
        if not self.function_profiles:
            return []
        
        sorted_functions = sorted(
            self.function_profiles.values(),
            key=lambda x: getattr(x, metric),
            reverse=True
        )
        
        return sorted_functions[:limit]
    
    def analyze_memory_usage(self) -> Dict[str, Any]:
        """分析内存使用情况"""
        if not self.is_profiling:
            # 手动收集内存信息
            gc.collect()  # 强制垃圾回收
            
            # 分析对象引用
            object_counts = {}
            for obj in gc.get_objects():
                obj_type = type(obj).__name__
                object_counts[obj_type] = object_counts.get(obj_type, 0) + 1
            
            # 获取内存最多的对象类型
            top_objects = sorted(object_counts.items(), key=lambda x: x[1], reverse=True)[:10]
            
            return {
                "total_objects": len(gc.get_objects()),
                "gc_counts": gc.get_count(),
                "top_object_types": top_objects,
                "memory_info": psutil.Process().memory_info()._asdict()
            }
        
        return {"message": "请先停止性能分析"}

# 1.2 性能装饰器
print(f"\n   🎨 1.2 性能装饰器:")

def performance_monitor(metric_name: str = None):
    """性能监控装饰器"""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            name = metric_name or f"{func.__module__}.{func.__name__}"
            
            start_time = time.time()
            start_memory = psutil.Process().memory_info().rss
            
            try:
                result = func(*args, **kwargs)
                success = True
                error = None
            except Exception as e:
                result = None
                success = False
                error = str(e)
                raise
            finally:
                end_time = time.time()
                end_memory = psutil.Process().memory_info().rss
                
                execution_time = end_time - start_time
                memory_delta = end_memory - start_memory
                
                metric = PerformanceMetric(
                    name=name,
                    metric_type=PerformanceMetricType.EXECUTION_TIME,
                    value=execution_time,
                    unit="seconds",
                    context={
                        "success": success,
                        "error": error,
                        "memory_delta": memory_delta,
                        "args_count": len(args),
                        "kwargs_count": len(kwargs)
                    }
                )
                
                print(f"性能监控 [{name}]: {execution_time:.4f}s, 内存变化: {memory_delta/1024/1024:.2f}MB")
            
            return result
        return wrapper
    return decorator

def memory_monitor(threshold_mb: float = 100.0):
    """内存监控装饰器"""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            process = psutil.Process()
            initial_memory = process.memory_info().rss / 1024 / 1024  # MB
            
            try:
                result = func(*args, **kwargs)
                
                final_memory = process.memory_info().rss / 1024 / 1024  # MB
                memory_increase = final_memory - initial_memory
                
                if memory_increase > threshold_mb:
                    print(f"⚠️ 内存警告 [{func.__name__}]: 内存增加 {memory_increase:.2f}MB (阈值: {threshold_mb}MB)")
                
                return result
                
            except Exception as e:
                print(f"内存监控错误 [{func.__name__}]: {e}")
                raise
        return wrapper
    return decorator

def cache_result(max_size: int = 128, ttl: int = 3600):
    """结果缓存装饰器"""
    def decorator(func):
        cache = {}
        cache_times = {}
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # 创建缓存键
            key = str(args) + str(sorted(kwargs.items()))
            
            current_time = time.time()
            
            # 检查缓存
            if key in cache:
                if current_time - cache_times[key] < ttl:
                    print(f"缓存命中 [{func.__name__}]")
                    return cache[key]
                else:
                    # 缓存过期，删除
                    del cache[key]
                    del cache_times[key]
            
            # 执行函数
            result = func(*args, **kwargs)
            
            # 缓存结果
            if len(cache) >= max_size:
                # 删除最旧的缓存项
                oldest_key = min(cache_times.keys(), key=cache_times.get)
                del cache[oldest_key]
                del cache_times[oldest_key]
            
            cache[key] = result
            cache_times[key] = current_time
            
            print(f"缓存设置 [{func.__name__}]")
            return result
        
        wrapper.cache_clear = lambda: cache.clear() or cache_times.clear()
        wrapper.cache_info = lambda: {
            'size': len(cache),
            'max_size': max_size,
            'ttl': ttl
        }
        
        return wrapper
    return decorator

def retry_on_failure(max_retries: int = 3, delay: float = 1.0, backoff: float = 2.0):
    """失败重试装饰器"""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            current_delay = delay
            
            for attempt in range(max_retries + 1):
                try:
                    if attempt > 0:
                        print(f"重试 [{func.__name__}] 第 {attempt} 次，延迟 {current_delay:.2f}s")
                        time.sleep(current_delay)
                        current_delay *= backoff
                    
                    result = func(*args, **kwargs)
                    
                    if attempt > 0:
                        print(f"重试成功 [{func.__name__}] 第 {attempt} 次")
                    
                    return result
                    
                except Exception as e:
                    last_exception = e
                    if attempt < max_retries:
                        print(f"重试失败 [{func.__name__}] 第 {attempt} 次: {e}")
                    else:
                        print(f"重试耗尽 [{func.__name__}]: {e}")
            
            raise last_exception
        return wrapper
    return decorator

print(f"   ✅ 性能分析工具完成")
print(f"      - PerformanceProfiler: 性能分析器")
print(f"      - performance_monitor: 性能监控装饰器")
print(f"      - memory_monitor: 内存监控装饰器")
print(f"      - cache_result: 结果缓存装饰器")

print(f"\n✅ 性能分析工具完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握性能分析和优化方法")
print(f"   ✓ 学会使用性能监控工具")
print(f"   ✓ 理解性能指标收集和分析")
print(f"   ✓ 能够建立基础的性能监控体系")

### 调试技巧与优化策略 [⭐⭐进阶]
**知识点说明**：调试技巧与优化策略是提升开发效率和系统性能的关键，为LangChain的AI项目提供全面的优化方案。

**学习要求**：
- 掌握高级调试技巧和工具
- 学会代码优化策略和模式
- 理解并发和异步优化
- 能够建立完整的优化体系

**案例要求**：
- 实现调试工具和日志系统
- 构建优化策略管理器
- 开发并发优化框架
- 验证点：能独立进行系统优化和调试

In [ ]:
print("\n🔍 调试技巧与优化策略:")
print("=" * 50)

# 2. 调试工具系统
print(f"📝 2. 调试工具系统:")

# 2.1 高级调试器
print(f"\n   🛠️ 2.1 高级调试器:")

class LogLevel(Enum):
    """日志级别"""
    DEBUG = "DEBUG"
    INFO = "INFO"
    WARNING = "WARNING"
    ERROR = "ERROR"
    CRITICAL = "CRITICAL"

@dataclass
class DebugEvent:
    """调试事件"""
    timestamp: datetime.datetime
    level: LogLevel
    message: str
    module: str
    function: str
    line_number: int
    context: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            "timestamp": self.timestamp.isoformat(),
            "level": self.level.value,
            "message": self.message,
            "module": self.module,
            "function": self.function,
            "line_number": self.line_number,
            "context": self.context
        }

class AdvancedDebugger:
    """高级调试器"""
    
    def __init__(self):
        self.events: List[DebugEvent] = []
        self.breakpoints: Dict[str, List[int]] = {}
        self.watch_variables: Dict[str, Any] = {}
        self.call_stack: List[Dict[str, Any]] = []
        self.is_debugging = False
        self.log_file: Optional[str] = None
    
    def enable_debugging(self, log_file: str = None):
        """启用调试模式"""
        self.is_debugging = True
        self.log_file = log_file
        print(f"调试模式已启用，日志文件: {log_file or '控制台'}")
    
    def disable_debugging(self):
        """禁用调试模式"""
        self.is_debugging = False
        print("调试模式已禁用")
    
    def log_event(self, level: LogLevel, message: str, context: Dict[str, Any] = None):
        """记录调试事件"""
        if not self.is_debugging:
            return
        
        # 获取调用栈信息
        frame = inspect.currentframe().f_back
        
        event = DebugEvent(
            timestamp=datetime.datetime.utcnow(),
            level=level,
            message=message,
            module=frame.f_globals.get('__name__', 'unknown'),
            function=frame.f_code.co_name,
            line_number=frame.f_lineno,
            context=context or {}
        )
        
        self.events.append(event)
        
        # 输出日志
        log_message = f"[{level.value}] {event.module}:{event.function}:{event.line_number} - {message}"
        
        if self.log_file:
            with open(self.log_file, 'a', encoding='utf-8') as f:
                f.write(f"{event.timestamp.isoformat()} {log_message}\n")
        else:
            print(log_message)
    
    def add_breakpoint(self, module: str, line_number: int):
        """添加断点"""
        if module not in self.breakpoints:
            self.breakpoints[module] = []
        
        if line_number not in self.breakpoints[module]:
            self.breakpoints[module].append(line_number)
            print(f"断点添加: {module}:{line_number}")
    
    def remove_breakpoint(self, module: str, line_number: int):
        """移除断点"""
        if module in self.breakpoints and line_number in self.breakpoints[module]:
            self.breakpoints[module].remove(line_number)
            print(f"断点移除: {module}:{line_number}")
    
    def watch_variable(self, name: str, value: Any):
        """监视变量"""
        old_value = self.watch_variables.get(name)
        self.watch_variables[name] = value
        
        if old_value is not None and old_value != value:
            self.log_event(
                LogLevel.DEBUG,
                f"变量变化: {name} = {value} (之前: {old_value})",
                {"variable": name, "old_value": old_value, "new_value": value}
            )
    
    def trace_function_call(self, func_name: str, args: tuple, kwargs: dict):
        """追踪函数调用"""
        self.log_event(
            LogLevel.DEBUG,
            f"函数调用: {func_name}",
            {
                "function": func_name,
                "args_count": len(args),
                "kwargs_keys": list(kwargs.keys())
            }
        )
        
        self.call_stack.append({
            "function": func_name,
            "timestamp": datetime.datetime.utcnow(),
            "args": args,
            "kwargs": kwargs
        })
    
    def trace_function_return(self, func_name: str, result: Any):
        """追踪函数返回"""
        if self.call_stack and self.call_stack[-1]["function"] == func_name:
            call_info = self.call_stack.pop()
            execution_time = (datetime.datetime.utcnow() - call_info["timestamp"]).total_seconds()
            
            self.log_event(
                LogLevel.DEBUG,
                f"函数返回: {func_name} (耗时: {execution_time:.4f}s)",
                {
                    "function": func_name,
                    "execution_time": execution_time,
                    "result_type": type(result).__name__
                }
            )
    
    def get_debug_summary(self) -> Dict[str, Any]:
        """获取调试摘要"""
        level_counts = {}
        for event in self.events:
            level_counts[event.level.value] = level_counts.get(event.level.value, 0) + 1
        
        return {
            "total_events": len(self.events),
            "level_distribution": level_counts,
            "breakpoints": dict(self.breakpoints),
            "watched_variables": len(self.watch_variables),
            "call_stack_depth": len(self.call_stack),
            "is_debugging": self.is_debugging
        }
    
    def export_events(self, filename: str):
        """导出调试事件"""
        events_data = [event.to_dict() for event in self.events]
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(events_data, f, indent=2, ensure_ascii=False)
        
        print(f"调试事件已导出到: {filename}")

# 2.2 优化策略管理器
print(f"\n   ⚡ 2.2 优化策略管理器:")

class OptimizationType(Enum):
    """优化类型"""
    ALGORITHM = "algorithm"  # 算法优化
    DATA_STRUCTURE = "data_structure"  # 数据结构优化
    MEMORY = "memory"  # 内存优化
    CPU = "cpu"  # CPU优化
    IO = "io"  # IO优化
    NETWORK = "network"  # 网络优化
    CACHE = "cache"  # 缓存优化
    CONCURRENCY = "concurrency"  # 并发优化

@dataclass
class OptimizationStrategy:
    """优化策略"""
    name: str
    description: str
    optimization_type: OptimizationType
    priority: int  # 1-10, 10最高
    expected_improvement: float  # 预期改进百分比
    implementation_cost: int  # 实现成本 1-10
    risk_level: int  # 风险等级 1-10
    applicable_modules: List[str]
    
    @property
    def roi_score(self) -> float:
        """投资回报率评分"""
        return (self.expected_improvement * self.priority) / (self.implementation_cost * self.risk_level)

class OptimizationManager:
    """优化管理器"""
    
    def __init__(self):
        self.strategies: List[OptimizationStrategy] = []
        self.applied_optimizations: List[Dict[str, Any]] = []
        self.performance_baseline: Optional[Dict[str, Any]] = None
        self.optimization_results: List[Dict[str, Any]] = []
    
    def add_strategy(self, strategy: OptimizationStrategy):
        """添加优化策略"""
        self.strategies.append(strategy)
        print(f"优化策略添加: {strategy.name} (ROI评分: {strategy.roi_score:.2f})")
    
    def set_baseline(self, performance_data: Dict[str, Any]):
        """设置性能基线"""
        self.performance_baseline = performance_data
        print("性能基线已设置")
    
    def get_recommended_strategies(self, limit: int = 5) -> List[OptimizationStrategy]:
        """获取推荐的优化策略"""
        sorted_strategies = sorted(
            self.strategies,
            key=lambda x: x.roi_score,
            reverse=True
        )
        
        return sorted_strategies[:limit]
    
    def apply_optimization(self, strategy: OptimizationStrategy, 
                           implementation_func: Callable) -> bool:
        """应用优化策略"""
        print(f"应用优化策略: {strategy.name}")
        
        start_time = time.time()
        
        try:
            # 执行优化
            implementation_func()
            
            execution_time = time.time() - start_time
            
            # 记录优化应用
            optimization_record = {
                "strategy_name": strategy.name,
                "applied_at": datetime.datetime.utcnow().isoformat(),
                "implementation_time": execution_time,
                "success": True
            }
            
            self.applied_optimizations.append(optimization_record)
            
            print(f"优化策略应用成功: {strategy.name} (耗时: {execution_time:.2f}s)")
            return True
            
        except Exception as e:
            print(f"优化策略应用失败: {strategy.name} - {e}")
            
            optimization_record = {
                "strategy_name": strategy.name,
                "applied_at": datetime.datetime.utcnow().isoformat(),
                "implementation_time": time.time() - start_time,
                "success": False,
                "error": str(e)
            }
            
            self.applied_optimizations.append(optimization_record)
            return False
    
    def measure_improvement(self, current_performance: Dict[str, Any]) -> Dict[str, float]:
        """测量性能改进"""
        if not self.performance_baseline:
            return {"error": "未设置性能基线"}
        
        improvements = {}
        
        for metric, baseline_value in self.performance_baseline.items():
            if metric in current_performance:
                current_value = current_performance[metric]
                
                if isinstance(baseline_value, (int, float)) and isinstance(current_value, (int, float)):
                    if baseline_value != 0:
                        improvement = ((baseline_value - current_value) / baseline_value) * 100
                        improvements[metric] = improvement
                    else:
                        improvements[metric] = 0.0
        
        return improvements
    
    def get_optimization_report(self) -> Dict[str, Any]:
        """获取优化报告"""
        total_strategies = len(self.strategies)
        applied_count = len([opt for opt in self.applied_optimizations if opt["success"]])
        failed_count = len([opt for opt in self.applied_optimizations if not opt["success"]])
        
        # 按类型统计策略
        type_counts = {}
        for strategy in self.strategies:
            opt_type = strategy.optimization_type.value
            type_counts[opt_type] = type_counts.get(opt_type, 0) + 1
        
        return {
            "total_strategies": total_strategies,
            "applied_optimizations": applied_count,
            "failed_optimizations": failed_count,
            "success_rate": applied_count / total_strategies if total_strategies > 0 else 0,
            "strategy_types": type_counts,
            "top_strategies": [
                {
                    "name": s.name,
                    "type": s.optimization_type.value,
                    "roi_score": s.roi_score,
                    "expected_improvement": s.expected_improvement
                }
                for s in self.get_recommended_strategies(3)
            ]
        }

# 2.3 并发优化框架
print(f"\n   🚀 2.3 并发优化框架:")

class ConcurrencyOptimizer:
    """并发优化器"""
    
    def __init__(self):
        self.thread_pool_size = multiprocessing.cpu_count()
        self.process_pool_size = multiprocessing.cpu_count()
        self.active_tasks: Dict[str, threading.Thread] = {}
        self.task_queue: queue.Queue = queue.Queue()
        self.results: Dict[str, Any] = {}
    
    def optimize_threading(self, func: Callable, data: List[Any], 
                           chunk_size: int = None) -> List[Any]:
        """线程优化"""
        if chunk_size is None:
            chunk_size = max(1, len(data) // self.thread_pool_size)
        
        # 分割数据
        chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]
        
        print(f"线程优化: {len(chunks)} 个块, {self.thread_pool_size} 个线程")
        
        results = []
        threads = []
        
        def worker(chunk):
            return [func(item) for item in chunk]
        
        # 创建并启动线程
        for chunk in chunks:
            thread = threading.Thread(target=worker, args=(chunk,))
            thread.start()
            threads.append(thread)
        
        # 等待所有线程完成
        for thread in threads:
            thread.join()
        
        print(f"线程优化完成")
        return results
    
    def optimize_processing(self, func: Callable, data: List[Any]) -> List[Any]:
        """进程优化"""
        print(f"进程优化: {len(data)} 个任务, {self.process_pool_size} 个进程")
        
        with multiprocessing.Pool(self.process_pool_size) as pool:
            results = pool.map(func, data)
        
        print(f"进程优化完成")
        return results
    
    def async_execute(self, func: Callable, *args, **kwargs) -> str:
        """异步执行"""
        task_id = str(uuid.uuid4())
        
        def async_worker():
            try:
                result = func(*args, **kwargs)
                self.results[task_id] = {"status": "success", "result": result}
            except Exception as e:
                self.results[task_id] = {"status": "error", "error": str(e)}
            finally:
                if task_id in self.active_tasks:
                    del self.active_tasks[task_id]
        
        thread = threading.Thread(target=async_worker)
        thread.start()
        self.active_tasks[task_id] = thread
        
        print(f"异步任务启动: {task_id}")
        return task_id
    
    def get_task_result(self, task_id: str, timeout: float = None) -> Optional[Dict[str, Any]]:
        """获取任务结果"""
        if task_id in self.active_tasks:
            # 任务还在运行中
            if timeout:
                self.active_tasks[task_id].join(timeout)
            
            if task_id in self.active_tasks:
                return {"status": "running"}
        
        return self.results.get(task_id)
    
    def get_concurrency_stats(self) -> Dict[str, Any]:
        """获取并发统计"""
        return {
            "thread_pool_size": self.thread_pool_size,
            "process_pool_size": self.process_pool_size,
            "active_tasks": len(self.active_tasks),
            "completed_tasks": len(self.results),
            "cpu_count": multiprocessing.cpu_count()
        }

print(f"   ✅ 调试工具系统完成")
print(f"      - AdvancedDebugger: 高级调试器")
print(f"      - OptimizationManager: 优化策略管理器")
print(f"      - ConcurrencyOptimizer: 并发优化器")
print(f"      - 支持完整的调试和优化流程")

print(f"\n✅ 调试技巧与优化策略完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握高级调试技巧和工具")
print(f"   ✓ 学会代码优化策略和模式")
print(f"   ✓ 理解并发和异步优化")
print(f"   ✓ 能够建立完整的优化体系")

## 📋 学习总结

### 知识清单达成情况

#### ✅ 已完成知识点

**8.5 性能优化与调试技巧 [⭐⭐进阶]**
- ✅ 掌握性能分析和优化方法
- ✅ 学会使用调试工具和性能监控
- ✅ 理解内存管理和并发优化
- ✅ 能够建立完整的性能调优体系

#### 🎯 核心技能掌握

**性能分析**
- ✅ CPU和内存性能分析
- ✅ 函数性能监控和统计
- ✅ 性能指标收集和分析
- ✅ 性能瓶颈识别和定位

**调试技巧**
- ✅ 高级调试器设计和实现
- ✅ 日志系统和事件追踪
- ✅ 断点和变量监视
- ✅ 调试信息导出和分析

**优化策略**
- ✅ 优化策略管理和评估
- ✅ 投资回报率分析
- ✅ 优化效果测量
- ✅ 风险评估和成本控制

#### 🚀 实践应用能力

**性能监控**
- ✅ 实时性能监控系统
- ✅ 性能装饰器设计
- ✅ 缓存和重试机制
- ✅ 内存泄漏检测

**系统优化**
- ✅ 并发和异步优化
- ✅ 算法和数据结构优化
- ✅ IO和网络优化
- ✅ 系统资源管理

#### 📊 与LangChain的关联

**AI性能优化**
- ✅ 支持LangChain应用的性能优化
- ✅ AI模型推理性能监控
- ✅ 机器学习管道优化
- ✅ 大规模并发AI请求处理

**工程化支持**
- ✅ 为LangChain提供性能调优方案
- ✅ 支持AI系统的实时监控
- ✅ 保障AI服务的高性能运行
- ✅ 实现AI组件的自动化优化

---

## 🎯 学习成果

通过本节学习，你已经掌握了：

1. **性能分析技术** - 理解性能分析的核心概念，能够使用专业工具进行系统性能分析
2. **调试工具开发** - 掌握高级调试器的设计和实现，能够建立完整的调试体系
3. **优化策略管理** - 学会优化策略的评估和管理，能够制定科学的优化计划
4. **并发优化技术** - 掌握多线程、多进程和异步编程的优化技巧

这些技能为LangChain等AI项目的性能保障提供了完整的解决方案，确保了系统的高性能和稳定性。接下来将继续学习代码重构与维护等工程实践技能。